In [ ]:
# Import Selenium and its sub libraries
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import re

In [ ]:
url="https://www.archdaily.com/"

In [ ]:
def extract_page_data(driver):
    #Get all projects
    projects=driver.find_elements("class name", "gridview__item")
    sleep(10)

    print(len(projects))

    for p in range(len(projects)):
          
            #Extracting title
            name = projects[p].find_element("class name","gridview__entry-title").text
            print(name)
        
            #Extraction of link
            link = projects[p].find_element("class name","gridview__content").get_attribute('href')
            #print(link)
        
                  
            #Add item to the respective lists
            name_list.append(name)
            link_list.append(link)
            sleep(2)  

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

#Step 1: identify search box
search_query = driver.find_element("xpath","//input[@placeholder='Search ArchDaily']")

#Step 2: search a topic
search_query.send_keys('MVRDV')
search_query.send_keys(Keys.RETURN)
#Delay
sleep(5)

#Select the project category
driver.find_element("xpath","/html/body/div[5]/div/div[1]/div[2]/nav/ul/li[2]/a").click()
sleep(6)

#Create empty lists to store data
name_list=[]
link_list=[]

#Archdaily does not load all the results at once. The results are loading while scrolling

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height


#Run the function to gather the data
extract_page_data(driver)


#Close the webpage
driver.quit()

In [ ]:
#Write the data to a csv file

import csv 
        
with open('archlinks.csv', 'w',newline='') as csvfile: 
    writer = csv.writer(csvfile) 
    for link in link_list:
        print(link)
        writer.writerow([link])

In [ ]:
## Iterate through the csv and download the images
#import library
import requests

driver = webdriver.Chrome()

#define a counter for naming 
i=0

#read the produced csv and iterate through the list
with open("archlinks.csv", 'r',encoding='utf-8') as file:
    csvreader = csv.reader(file)    
    for url in csvreader:       
                
        #open each link 
        driver.get(url[0])
        sleep(2)
        
        #find the image link
        imglink = driver.find_element("tag name","source").get_attribute('srcset') 
        
        
        print(imglink)
        i += 1    
        
        response = requests.get(imglink)
        #name the images
        imagename="image" + str(i)+".jpg"
        #save the images
        with open(imagename, "wb") as f:
            f.write(response.content)
       
    
#Close the webpage
driver.quit()